### Necessary Imports

In [46]:
import numpy as np
import pandas as pd
from bs4 import BeautifulSoup
import requests

### Use Beautiful Soup to pull all contents of the web-page

In [31]:
page = requests.get('https://en.wikipedia.org/wiki/UEFA_Cup_and_Europa_League_records_and_statistics')
soup = BeautifulSoup(page.content,'html.parser')

### Find the necessary table on the page. First let us consider the finalist data.

In [32]:
tables = soup.findAll("table")

### Build Table Header

In [33]:
table_header = tables[1].findAll("tr")
header_list=[]
for i in range(1,len(table_header[0])):
    if(i==1):
        header_list.append(table_header[0].contents[i].contents[0].lstrip().rstrip())
    elif (i%2 != 0):
        header_list.append(table_header[0].contents[i].contents[0].lstrip().rstrip())

### Create an empty data frame with all the columns

In [34]:
el_final_record = pd.DataFrame(columns=header_list)

### Consume all the row element columns and populate the data frame for the finalists

In [35]:
rows = tables[1].findAll("tr")
x = rows[1].contents[9].findAll("a")
for club in range(1,len(rows)):
    club_name = rows[club].contents[1].contents[2].contents[0]
    titles = rows[club].contents[3].contents[0].rstrip()
    runners = rows[club].contents[3].contents[0].rstrip()
    x = rows[club].contents[7].findAll("a")
    win_list = []
    for i in range(0,len(x)):
        win_list.append(x[i].contents[0])
    y = rows[club].contents[9].findAll("a")
    runner_list = []
    for j in range(0,len(y)):
        runner_list.append(y[j].contents[0])
    el_final_record.loc[club-1] = [club_name,titles,runners,win_list,runner_list]

### Find the data for the semi-finalists and build a seperate data frame

In [36]:
semi_table_hdr = tables[8].findAll("tr")
semi_hdr_list = []
for i in range(1,len(semi_table_hdr[0])):
    if (i%2 != 0):
        semi_hdr_list.append(semi_table_hdr[0].contents[i].contents[0].lstrip().rstrip())

### Consume all the row element columns and populate the data frame for the finalists

In [37]:
el_semi_record = pd.DataFrame(columns=semi_hdr_list)
rows = tables[8].findAll("tr")
for club in range(1,len(rows)):
    club_name = rows[club].contents[1].contents[2].contents[0]
    number_of_semis = rows[club].contents[3].contents[0].rstrip()
    x = rows[club].contents[5].findAll("a")
    years = []
    for i in range(0,len(x)):
        years.append(x[i].contents[0])
    el_semi_record.loc[club-1] = [club_name,number_of_semis,years]

In [38]:
season_dict={}
for i in range(0,len(el_final_record)):
    club_name = el_final_record['Club'][i]
    winning_list = el_final_record[el_final_record.Club == club_name]['Years won'].tolist()
    for i in range(0,len(winning_list)):
            season_list = winning_list[i]
            if(len(season_list)!=0):
                for i in season_list:
                    season_dict[i] = [club_name]

### Using the other website to get the runner-up details

In [39]:
supp_page = requests.get('https://en.wikipedia.org/wiki/List_of_UEFA_Cup_and_Europa_League_finals')
supp_soup = BeautifulSoup(supp_page.content,'html.parser')
supp_tables = supp_soup.findAll("table")
x=supp_tables[2].find_all("th")
hdr_list = ['Season','Runners-Up']
runners = pd.DataFrame(columns=hdr_list)
season_list=[]
for i in range(8,56):
    season = (x[i].contents[0].contents[0])
    if len(season) > 7:
        season_list.append(season[-4:])
    else:
        season_list.append(season[0:2]+season[-2:])
con = supp_tables[2].find_all("td")
runners.Season = season_list
runners_list=[]
for i in range(3,264,10):
    runners_list.append(con[i].contents[0])
for i in range(270,414,7):
    runners_list.append(con[i].contents[0])
runner_up=[]
for i in range(0,len(runners_list)):
    runner_up.append(runners_list[i].contents[0])
runners['Runners-Up'] = runner_up
for i in range(0,len(runners)):
    season_dict[runners['Season'][i]].append(runners['Runners-Up'][i])

### Now add the other semi-finalists from the semi finalist data frame to complete the dataset

In [40]:
teams = el_semi_record['Team']
for team in teams:
    season_list=[]
    team_list = el_semi_record[el_semi_record['Team']==team]['Years']
    team_index = team_list.index[0]
    season_list = team_list[team_index]
    for season in season_list:
        if(team!=season_dict[season][0]):
            if(team!=season_dict[season][1]):
                season_dict[season].append(team)

### Clean the data-set to remove duplication caused due to the use of different names for the same team

In [41]:
season_dict_items = season_dict.items()
for i in season_dict_items:
    if (len(i[1])>4):
        if((season_dict[i[0]][2][0:5]==season_dict[i[0]][0][0:5]) or (season_dict[i[0]][2][0:5]==season_dict[i[0]][1][0:5])):
            season_dict[i[0]].remove(season_dict[i[0]][2])
        elif((season_dict[i[0]][3][0:5]==season_dict[i[0]][0][0:5]) or (season_dict[i[0]][3][0:5]==season_dict[i[0]][1][0:5])):
            season_dict[i[0]].remove(season_dict[i[0]][3])
        elif((season_dict[i[0]][4][0:5]==season_dict[i[0]][0][0:5]) or (season_dict[i[0]][4][0:5]==season_dict[i[0]][1][0:5])):
            season_dict[i[0]].remove(season_dict[i[0]][4])

### Clean the data-set to remove duplication caused due to the use of different names for the same team

In [42]:
for i in season_dict_items:
    if (len(i[1])>4):
        if((season_dict[i[0]][2][-6:]==season_dict[i[0]][0][-6:]) or (season_dict[i[0]][2][-6:]==season_dict[i[0]][1][-6:])):
            season_dict[i[0]].remove(season_dict[i[0]][2])
        elif((season_dict[i[0]][3][-6:]==season_dict[i[0]][0][-6:]) or (season_dict[i[0]][3][-6:]==season_dict[i[0]][1][-6:])):
            season_dict[i[0]].remove(season_dict[i[0]][3])
        elif((season_dict[i[0]][4][-6:]==season_dict[i[0]][0][-6:]) or (season_dict[i[0]][4][-6:]==season_dict[i[0]][1][-6:])):
            season_dict[i[0]].remove(season_dict[i[0]][4])

### Convert the list of dictionaries into a self-contained data-frame which has winners, runner-ups and finalists for each season

In [43]:
europa_league_data = pd.DataFrame(list(season_dict.items()))
europa_league_data.columns = ['Season','Teams']
winner_list=[]
runner_list=[]
sf1_list=[]
sf2_list=[]
for i in range(0,len(europa_league_data)):
    winner_list.append(europa_league_data.Teams[i][0])
    runner_list.append(europa_league_data.Teams[i][1])
    sf1_list.append(europa_league_data.Teams[i][2])
    sf2_list.append(europa_league_data.Teams[i][3])
europa_league_data['Winners'] = winner_list
europa_league_data['Runner Up'] = runner_list
europa_league_data['SF1'] = sf1_list
europa_league_data['SF2'] = sf2_list
europa_league_data = europa_league_data.drop(columns='Teams')
europa_league_data = europa_league_data.sort_values('Season')
europa_league_data = europa_league_data.set_index('Season')

In [45]:
europa_league_data.to_csv('Europa_League_Data.csv',index=False)